# Toys
The goal is to experiment the capabilities of RL. Document each usefull functionalitiy.
This code aim to try:
- different environement
- different agent
- different hardware
- Monitoring learning
- evaluating and comparing
- Complete a table with the KPIs

## Dependecies
- [Gymansium](https://gymnasium.farama.org/): Environments
- stable-baseline3: Agents 

In [9]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 11.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0


In [ ]:
!pip install stable-baselines3[extra]
!pip install panda-gym
!pip install gymnasium[box2d][classic-control]
!pip install tqdm
!pip install tensorboard
!pip install ipywidgets
!pip install onnx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 KB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 KB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 720.1 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 KB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━

In [1]:
import gymnasium as gym
from gymnasium.wrappers import HumanRendering

import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
from stable_baselines3.common.logger import HParam
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.policies import BasePolicy
from stable_baselines3 import DQN
from stable_baselines3 import A2C


from matplotlib import pyplot as plt
import numpy as np
import os
import shutil
import torch
import time
import onnx
import onnxruntime as ort
from typing import Tuple


from tqdm import tqdm

## Device

In [2]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


## Setup

### Setup agent

In [9]:
policy = 'MlpPolicy' # Configure
# 'MlpPolicy'
# 'CnnPolicy'
# 'MultiInputPolicy'

verbose = 0

name = 'dqn_cartpole_v3' # Configure

path = f"./{name}/model/{name}"

tensorboard_log = f"./{name}/t_logs/"
tb_log_name = 'runs'

log_dir = f"./{name}/logs/"

onnx_model_path = f"./{name}/onnx/{name}.onnx"

### Setup environment

In [17]:
env_id = 'CartPole-v1' # Configure
# 'LunarLander-v2'
# 'CartPole-v1'
# 'PandaReachDense-v3'

max_episode_steps = 500

NORMALIZE = False

render_mode = 'rgb_array'
# 'human'
# 'rgb_array'

def make_env():
    e = gym.make(env_id, max_episode_steps=max_episode_steps, render_mode=render_mode)
    e = Monitor(e)
    return e

num_envs = 4

env_path = f'{name}/env/vec_normalize.pkl'

## Environment

### Make single environment

In [15]:
env = gym.make(env_id,max_episode_steps, render_mode=render_mode)

### Create vectorize environment

In [5]:

env = DummyVecEnv([make_env for _ in range(num_envs)])

# env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

# env_dir = os.path.dirname(env_path)
# if not os.path.exists(env_dir):
#     os.makedirs(env_dir)  # 
# env.save(env_path)

### Load vectorize environment

In [ ]:
env = VecNormalize.load(env_path, DummyVecEnv([make_env for _ in range(num_envs)]))


## Monitoring

In [6]:
class TensorboardCallback(BaseCallback):
    
    def __init__(self, verbose=0):
        super().__init__(verbose)

    def _on_step(self):
        exploration_rate = self.model.exploration_rate
        learning_rate = self.model.learning_rate
        self.logger.record("exploration_rate", exploration_rate)
        self.logger.record("learning_rate", learning_rate)
        return True
    
class HParamCallback(BaseCallback):

    def _on_training_start(self):
        hparam_dict = {
            "algorithm":self.model.__class__.__name__,
            "learning rate": self.model.learning_rate,
            "gamma": self.model.gamma
        }
        metric_dict = {
            "rollout/ep_len_mean":0,
            "train/value_loss": 0.0
        }
        self.logger.record(
            "hparams",
            HParam(hparam_dict, metric_dict),
            exclude=("stdout","log","json","csv")
        )

    def _on_step(self):
        return True
    
class SaveOnBestTrainingRewardCallback(BaseCallback):
    
    def __init__(self, check_freq:int, log_dir:str, path:str,verbose = 1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = path
        self.best_mean_reward = -np.inf

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), "timesteps")
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose >= 1:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose >= 1:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

        return True
        
class SaveOnStep(BaseCallback):
    def __init__(self, steps: int, path: str, verbose: int = 0):
        super().__init__(verbose)
        self.steps = steps
        self.save_path = path

    def _on_step(self) -> bool:
        # Check if the current step matches the saving frequency
        if self.n_calls % self.steps == 0:
            # Save model with the current timestep in the filename
            if self.verbose > 0:
                print(f"Saving model at step {self.n_calls} to {self.save_path}")
            self.model.save(self.save_path)
        return True

callbacks = CallbackList([
    #TensorboardCallback(),
    #HParamCallback(),
    #SaveOnBestTrainingRewardCallback(1e4,log_dir,path)
    SaveOnStep(1e4, path, verbose=1)
])



### Debug

In [8]:
print(env)

## Agents

### Create agent

In [ ]:
model = DQN( # Configure
    policy, 
    env, 
    learning_rate=1e-4,
    exploration_initial_eps=0.9,
    exploration_fraction=0.3,
    verbose=verbose,
    device=device,
    tensorboard_log=tensorboard_log)
# DQN
# A2C
# DDPG
# PPO


#model.save(path)

/home/dorvan/Documents/dorvan/Babydoll/Produits/Babybot-01/Informatic-01/Learn RL/hostEnv/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'dqn_cartpole_v3/model' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


### Load agent

In [8]:

model = DQN.load(path,env,tensorboard_log=tensorboard_log,device=device)
# DQN
# A2C
# DDPG

### Train agent

In [11]:
total_timesteps = 1e6
model = DQN.load(path,env,tensorboard_log=tensorboard_log,device=device)
model.learn(
    total_timesteps, 
    progress_bar=True, 
    reset_num_timesteps=False, 
    tb_log_name=tb_log_name,
    callback=callbacks
    )

model.save(path)

Output()

Saving model at step 10000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

Saving model at step 20000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

Saving model at step 30000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

Saving model at step 40000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

...

Saving model at step 210000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

Saving model at step 220000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

Saving model at step 230000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

Saving model at step 240000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

Saving model at step 250000 to ./dqn_cartpole_v3/model/dqn_cartpole_v3

### Monitor 

In [ ]:
!tensorboard --logdir ./logs/tesorboard_log_{name}

### Evaluate agent

In [13]:
eval_env = VecNormalize.load(env_path, DummyVecEnv([make_env]))
eval_env.training = False
eval_env.gamma = 0.0
eval_env.norm_reward = True

n_eval_episodes = 10

mean_reward, std_reward = evaluate_policy(model,eval_env,n_eval_episodes)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=10.20 +/- 0.9797958971132711


### Display agent

In [12]:
def make_display_env():
    e = gym.make(env_id, max_episode_steps=max_episode_steps, render_mode='human')
    e = Monitor(e)
    return e

# vec normalize env
#display_env = VecNormalize.load(env_path, DummyVecEnv([make_display_env]))

# vec env
display_env = DummyVecEnv([make_display_env])

# simple env
#display_env = gym.make(env_id, max_episode_steps=max_episode_steps,render_mode='human') #HumanRendering(env)

In [13]:


for ep in range(3):

    # obs, info = display_env.reset() # Simple Env
    obs = display_env.reset()
    while True:
        action, _states = model.predict(obs)

        #obs, reward, terminated, truncated, info = display_env.step(action) # Unwrapped Env
        obs, reward, terminated, info = display_env.step(action)
        terminated = terminated[0]
        truncated = info[0].get('TimeLimit.truncated', False)

        if terminated or truncated:
            break

    time.sleep(0.5)



## Convert to ONNX

### Save

In [14]:
class OnnxableSB3Policy(torch.nn.Module):
    def __init__(self, policy: BasePolicy):
        super().__init__()
        self.policy = policy

    def forward(self, observation: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
     
        return self.policy(observation, deterministic=True)
    
onnx_policy = OnnxableSB3Policy(model.policy)
observation_size = model.observation_space.shape
dummy_input = torch.randn(1, *observation_size)
os.makedirs(f'./{name}/onnx', exist_ok=True)
torch.onnx.export(
    onnx_policy,
    dummy_input,
    onnx_model_path,
    opset_version=17,
    input_names=["input"],
)

### Load onnx

In [15]:

onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

observation = np.zeros((1, *observation_size)).astype(np.float32)
ort_sess = ort.InferenceSession(onnx_model_path)
actions = ort_sess.run(None, {"input": observation})[0]

print(action)

# Check that the predictions are the same
with torch.no_grad():
    print(model.policy(torch.as_tensor(observation), deterministic=True))

[1]
tensor([0])


### Display

In [20]:

if NORMALIZE :
    vec_normalize = VecNormalize.load(env_path, DummyVecEnv([make_env for _ in range(num_envs)]))

onnx_env = gym.make(env_id, render_mode='human')

for e in range(5):

    obs, info = onnx_env.reset()

    done = False
    total_reward = 0

    while not done:
        # Preprocess observation to match ONNX input shape
        # Ensure it's a batch (e.g., [1, 4] for CartPole)
        #obs_input = np.expand_dims(obs, axis=0).astype(np.float32)
        
        if NORMALIZE : 
            obs = vec_normalize.normalize_obs(obs)

        obs = np.expand_dims(obs, axis=0).astype(np.float32)

        # Get action probabilities from the ONNX model
        action = ort_sess.run(None, {"input": obs})
        action = action[0].item()
       
        #action = np.expand_dims(action, axis=0)
        
        # Step in the environment
        obs, reward, done, truncated, info = onnx_env.step(action)
        total_reward += reward

    print(f"Total reward: {total_reward}")


Total reward: 190.0
Total reward: 200.0
Total reward: 231.0
Total reward: 209.0
Total reward: 211.0
